In [1]:
# Import required libraries

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
import json
import os

# You can use your own api key

model = ChatOpenAI(
    model_name="gpt-3.5-turbo",  # or "gpt-4" for more advanced capabilities
    temperature=0.1,
)

In [2]:
# Obtain job description
from job_description import JOB_DESCRIPTION
job_description = JOB_DESCRIPTION

In [3]:
# Open and load the JSON file that used for template
with open('resume-prompt.json', 'r') as file:
    json_prompt = json.load(file)
raw_resume_text = json.dumps(json_prompt, indent=2).replace("{", "{{").replace("}", "}}")


In [4]:
# import the examples needed for few shot prompting
from examples import RESUME_EXAMPLES

# Example formatting for few-shot examples
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=RESUME_EXAMPLES,
)

In [5]:
# Create the full prompt with few-shot examples included
full_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional resume optimizer that helps improve resumes for ATS systems."),
    few_shot_prompt,  # Include the few-shot examples as a message
    ("human", """
Below is a flawed or unstructured resume, a job description the candidate is applying to, and some helpful context from other successful resumes.

Use the job description and context to emphasize relevant skills and experiences. Your job is to rewrite and optimize the resume in valid JSON format aligned with ATS standards.

--- RAW RESUME ---
{raw_resume}

--- JOB DESCRIPTION ---
{job_description}

Please generate the optimized JSON resume:
""")
])

# Full chain with LLM and parser
resume_optimizer_chain = full_prompt | model | StrOutputParser()



In [6]:
# Invoke with real resume + JD
optimized_resume_json = resume_optimizer_chain.invoke({
    "raw_resume": raw_resume_text,
    "job_description": job_description
})

# Print or save the result
print(optimized_resume_json)

{
  "$schema": "https://raw.githubusercontent.com/jsonresume/resume-schema/v1.0.0/schema.json",
  "basics": {
    "summary": "Experienced data scientist with a background in applied information theory and optimization. Skilled in developing algorithms, analyzing data trends, and communicating insights to stakeholders. Strong mathematical and statistical expertise with a creative and curious mindset."
  },
  "work": [
    {
      "highlights": [
        "Developed algorithm for copyright infringement detection in music industry",
        "Won Techcrunch Disrupt competition",
        "Optimized algorithm achieving world record for Weisman Scores"
      ]
    }
  ],
  "skills": [
    {
      "name": "Data Science",
      "keywords": [
        "Data Mining",
        "Mathematics",
        "Statistical Analysis",
        "Pattern Recognition",
        "Predictive Modeling",
        "Excel",
        "PowerPoint",
        "Tableau",
        "SQL",
        "Python",
        "Java"
      ]
    

In [7]:
# Save to a file
with open("optimized_resume.json", "w") as f:
    f.write(optimized_resume_json)